# ECDH - Elliptic Curve Diffie-Hellman

In [376]:
class ECDH:
    def __init__(self, n):
        self.genCurve(n)
        
    def genCurve(self, n):
        K = GF(2^n)
        while True:
            b=0
            while b==0: #E cannot be a singular curve
                b = K.random_element()
            E = EllipticCurve(K,[1,1,0,0,b])
            e_ord = E.order()
            e_fact = list(factor(e_ord))[-1][0]
            if e_fact >= (2^(n-1)):
                while True:
                    P = E.random_element()
                    p_ord = P.order()
                    if p_ord > e_fact:
                        h = Integer(p_ord/e_fact)
                        Q = h * P
                        self.Q = Q
                        self.N = e_fact
                        return

    def genPrivPubKeyPair(self):
        k = randint(0, self.N-1)
        kQ = k*self.Q
        return k,kQ

    def genSharedKey(self, k, sQ):
        return k*sQ

## Alice

In [377]:
def Alice(conn, ecdh):
    k, kQ = ecdh.genPrivPubKeyPair()
    conn.send(kQ)
    sQ = conn.recv()
    sKey = ecdh.genSharedKey(k, sQ)
    print("[Alice] Shared key: " + str(sKey))

## Bob

In [378]:
def Bob(conn, ecdh):
    s, sQ = ecdh.genPrivPubKeyPair()
    kQ = conn.recv()
    conn.send(sQ)
    sKey = ecdh.genSharedKey(s, kQ)
    print("[Bob] Shared key: " + str(sKey))

In [379]:
from multiprocessing import Process, Pipe
from getpass import getpass

class Connection:
    def __init__(self, left, right, timeout=None):
        left_end, right_end = Pipe()
        self.timeout = timeout
        ecdh = ECDH(5)
        self.lproc = Process(target = left, args=(left_end, ecdh))
        self.rproc = Process(target = right, args=(right_end, ecdh))
        self.left = lambda : left(left_end)
        self.right = lambda : right(right_end)
        
    def auto(self, proc=None):
        if proc == None:
            self.lproc.start()
            self.rproc.start()
            self.lproc.join(self.timeout)
            self.rproc.join(self.timeout)
        else:
            proc.start()
            proc.join(self.timeout)
    def manual(self):
        self.left()
        self.right()

In [380]:
Connection(Alice, Bob, timeout=10).auto()

[Bob] Shared key: (z5^3 + z5^2 : z5^4 + z5^3 : 1)
[Alice] Shared key: (z5^3 + z5^2 : z5^4 + z5^3 : 1)
